In [ ]:
import concurrent.futures
from tqdm.notebook import tqdm

def evaluate_fitness(member):
    processes, servers = member
    fitness = 0
    for server in servers:
        execution_time = 0
        for process in server.workload:
            execution_time += process.length / (server.core_speed * server.num_cores)
        fitness += execution_time
    return fitness

def get_fittest_indexes(fitnesses, population_size, elitism_rate=0.1):
    fittest_indexes = sorted(range(len(fitnesses)), key=lambda x: fitnesses[x])[:int(population_size * elitism_rate)]
    return fittest_indexes

def create_offspring(fittest, population_size, tournament_size=3):
    offspring = []
    while len(offspring) < population_size:
        parent1 = max(random.sample(fittest, tournament_size), key=lambda x: evaluate_fitness(x))
        parent2 = max(random.sample(fittest, tournament_size), key=lambda x: evaluate_fitness(x))
        child = breed(parent1, parent2)
        offspring.append(child)
    return offspring

def mutate(member, mutation_rate=0.1):
    processes, servers = member
    if random.random() < mutation_rate:
        # Choose a random process and assign it to a random server
        process = random.choice(processes)
        server = random.choice(servers)
        process_index = processes.index(process)
        processes[process_index] = None
        server.workload.append(process)
    return [processes, servers]

def mutate_offspring(offspring, mutation_rate=0.1):
    for i in range(len(offspring)):
        offspring[i] = mutate(offspring[i], mutation_rate)
    return offspring

def breed(parent1, parent2):
    # Combine the process lists of the parents and shuffle them to create the child
    child_processes = parent1[0] + parent2[0]
    random.shuffle(child_processes)
    return [child_processes, parent1[1]]

def assign_processes(processes, servers, population_size=100, n_iter=100, elitism_rate=0.1, tournament_size=3, mutation_rate=0.1):
    # Initialize the population with randomly assigned sets of processes to servers
    population = []
    for _ in range(population_size):
        population.append([random.sample(processes, k=len(processes)), servers])

    best_fitness = float('inf')
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in tqdm(range(n_iter)):
            # Evaluate the fitness of each member of the population in parallel
            fitnesses = list(executor.map(evaluate_fitness, population))

            # Select the fittest members of the population for reproduction
            fittest_indexes = get_fittest_indexes(fitnesses, population_size, elitism_rate)
            fittest = [population[i] for i in fittest_indexes]

            # Keep track of the best fitness
            best_fitness = min(best_fitness, min(fitnesses))

            # Breed new members by combining the process lists of the fittest members
            offspring = create_offspring(fittest, population_size, tournament_size)

            # Add some random mutations to the process lists of the new members
            offspring = mutate_offspring(offspring, mutation_rate)

            # Replace the current population with the offspring
            population = offspring
            if best_fitness == min(fitnesses):
                break
    fittest_index = min(range(len(fitnesses)), key=lambda x: fitnesses[x])
    return population[fittest_index]

In [ ]:
from joblib import Parallel, delayed

def assign_processes_joblib(processes, servers, population_size=100, n_iter=100, elitism_rate=0.1, tournament_size=3, mutation_rate=0.1, n_jobs = -1):
    population = []
    for _ in range(population_size):
        population.append([random.sample(processes, k=len(processes)), servers])
    
    best_fitness = float('inf')
    for i in tqdm(range(n_iter)):
        fitnesses = Parallel(n_jobs=n_jobs)(delayed(evaluate_fitness)(member) for member in population)
        fittest_index